## Algo to start a speculative Optiontrade with Riskmanagment and Profit-taking


A call option is choosen to jump on an expected price-move of a stock.
The order is submitted as OCA-Braketed-Order

Please be aware, that this algo is not implementing a profitable trading strategy. You will loose money if you use it in production.


First, load ib-ruby environment

In [1]:
require_relative '../../lib/iruby_init.rb'
require 'rover-df'

----> G    points to the Gateway-Instance
----> C    points to the Connection-Instance
----> Multible accounts detected
----> G.clients  points to the User-Accounts
Namespace is IB !


true

Ensure that a Demo-Account is used

In [2]:
# raises an error and stops the script unless a paper trading account is used
exit unless G.clients.first.test_environment?

# initialize a proper client regardles of the type of account used
the_client =  G.clients.last; nil

-- 

Apple is the underlying. 

-- 

In [3]:
aapl =  Stock.new( symbol: :aapl ).verify.first
puts aapl.as_table
puts "Market-price: #{ref_price = aapl.market_price}"
# in case that no market-price is transmitted ( happens occasionally if the code is run on a paper account )
ref_price ||=  150; nil   # suppress output

┌───────┬────────┬────────┬──────────┬────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│       │ symbol │ con_id │ exchange │ expiry │ multiplier │ trading-class │ right │ strike │ currency │
╞═══════╪════════╪════════╪══════════╪════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Stock │ AAPL   │ 265598 │  SMART   │        │            │      NMS      │       │        │   USD    │
└───────┴────────┴────────┴──────────┴────────┴────────────┴───────────────┴───────┴────────┴──────────┘
Market-price: 148.48


Get out of the money options,  do not try ask for a market-price again. Use the ref price instead.

In [4]:
apple_options =  aapl.otm_options right: :call, ref_price: ref_price
puts "*"*20
puts "OTM Strikes"
puts apple_options.keys.join(" . ")

********************
OTM Strikes
148.0 . 149.0 . 150.0 . 152.5 . 155.0 . 157.5


Using the second (regular) expiry of the third otm-strike

In [5]:
the_strike = apple_options.keys.at 2
puts apple_options[the_strike].as_table

#####  This is the subset of the Option-Chain to select a candidate to trade
aapl_base =  apple_options[the_strike]

#### fetch the option parameters 
aapl_base.map{|x| Thread.new { x.request_greeks } }.join
sleep 1
nil # suppress output last statement

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ AAPL   │ 456051961 │   CBOE   │ 20210716 │    100     │     AAPL      │ call  │  150.0 │   USD    │
│ Option │ AAPL   │ 479433110 │   CBOE   │ 20210820 │    100     │     AAPL      │ call  │  150.0 │   USD    │
│ Option │ AAPL   │ 437882413 │   CBOE   │ 20210917 │    100     │     AAPL      │ call  │  150.0 │   USD    │
│ Option │ AAPL   │ 471892407 │   CBOE   │ 20211015 │    100     │     AAPL      │ call  │  150.0 │   USD    │
│ Option │ AAPL   │ 490643460 │   CBOE   │ 20211119 │    100     │     AAPL      │ call  │  150.0 │   USD    │
│ Option │ AAPL   │ 494194120 │   CBOE   │ 20211217 │    100     │     AAPL      │ call  │  150.0 │   USD    │
│

Repeat the last statement if the Greek-Table does not fill

In [12]:
puts aapl_base.greek.as_table

┌────────────────────────────┬─────────┬────────────┬──────────┬─────────┬─────────┬─────────┬─────────┐
│ Greeks                     │ price   │ impl. vola │ dividend │ delta   │ gamma   │ vega    │ theta   │
╞════════════════════════════╪═════════╪════════════╪══════════╪═════════╪═════════╪═════════╪═════════╡
│ call  AAPL 20210716  150.0 │    0.10 │       0.22 │     0.00 │    0.14 │    0.16 │    0.01 │   -0.10 │
│ call  AAPL 20210820  150.0 │    4.16 │       0.27 │     0.22 │    0.46 │    0.03 │    0.19 │   -0.07 │
│ call  AAPL 20210917  150.0 │    5.51 │       0.25 │     0.22 │    0.48 │    0.03 │    0.25 │   -0.05 │
│ call  AAPL 20211015  150.0 │    6.80 │       0.25 │     0.22 │    0.49 │    0.02 │    0.30 │   -0.04 │
│ call  AAPL 20211119  150.0 │    8.60 │       0.27 │     0.44 │    0.50 │    0.02 │    0.35 │   -0.04 │
│ call  AAPL 20211217  150.0 │    9.54 │       0.27 │     0.44 │    0.51 │    0.02 │    0.38 │   -0.03 │
│ call  AAPL 20220121  150.0 │   10.62 │       0.27 │  

Now, select the appropiate option.
  
For simplicity, this demo just picks the third option from the list

In [13]:
apple_option =  aapl_base.at 2
puts apple_option.as_table
puts "Ref-price of the Option:  #{apple_option.greek.option_price.round(4)}"

┌────────┬────────┬───────────┬──────────┬──────────┬────────────┬───────────────┬───────┬────────┬──────────┐
│        │ symbol │ con_id    │ exchange │ expiry   │ multiplier │ trading-class │ right │ strike │ currency │
╞════════╪════════╪═══════════╪══════════╪══════════╪════════════╪═══════════════╪═══════╪════════╪══════════╡
│ Option │ AAPL   │ 437882413 │   CBOE   │ 20210917 │    100     │     AAPL      │ call  │  150.0 │   USD    │
└────────┴────────┴───────────┴──────────┴──────────┴────────────┴───────────────┴───────┴────────┴──────────┘
Ref-price of the Option:  5.5095


#### Setup the Trade

* The Entry will be  Ref-price - 3 %

* The Stop-Price is -50 % of the reference price

* Profit-Taking happens on + 30 % from the reference price

In [14]:
adjusted_price = -> (rel) do 
   rel =  rel.to_f / 100 if rel.abs > 1
  # adjust round to a proper value if the order is not accepted due to improper price increment
   (apple_option.greek.option_price + (apple_option.greek.option_price.to_f * rel )).round(1)
end

#<Proc:0x000055e5fd0b25a0 (irb):0 (lambda)>

Place the prepared orders

In [15]:
C.send_message :RequestGlobalCancel
G.update_orders
the_client =  G.clients.last
#exit unless G.clients.orders.flatten.empty?

puts "market_price: #{apple_option.market_price}"

puts "removed pending orders"
puts "\nestablished orders:"
buy_order = IB::Limit.order :size => 10,
                            :price => adjusted_price[ -3 ],
                            :action => :buy,
                            :algo_strategy => '',
                            :transmit => true

the_client.place contract: apple_option, order: buy_order

puts buy_order.to_human

stop_order = IB::SimpleStop.order :size => 10,
                            :price => adjusted_price[ -50 ],
                            :action => :sell,
                            :parent_id =>buy_order.local_id

the_client.place contract: apple_option, order: stop_order

puts stop_order.to_human

profit_order = IB::Limit.order  :size => 10,
                          :price => adjusted_price[ 30 ] ,
                          :action => :sell,
                          :parent_id => buy_order.local_id

the_client.place contract: apple_option, order: profit_order

puts profit_order.to_human

market_price: 5.62
removed pending orders

established orders:
<Order: LMT GTC buy 10  @ 5.3 New #4/ from 48666/DU4035279>
<Order: STP GTC sell 10  @ New /2.8#5/ from 48666/DU4035279>
<Order: LMT GTC sell 10  @ 7.2 New #6/ from 48666/DU4035279>


Request all active orders, update the bookkeeping on the clients and print a summary

In [17]:
G.update_orders
puts the_client.orders.to_human

<Order: LMT GTC sell 10.0  @ 7.2 PreSubmitted #6/1549732467 from 48666/DU4035279>
<Order: STP GTC sell 10.0  @ 0.0 PreSubmitted /2.8#5/1549732466 from 48666/DU4035279>
<Order: LMT GTC buy 10.0  @ 5.3 Submitted #4/1549732465 from 48666/DU4035279>


In [11]:
C.send_message :RequestGlobalCancel

true